In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os

import cftime
import numpy as np
import xarray as xr

import config
import util

In [3]:
catalog = util.curate_flux_products().open_catalog()
catalog

flux_products-catalog-local:
  args:
    path: catalogs/flux_products-catalog-local.yml
  description: Flux products for transport modeling
  driver: intake.catalog.local.YAMLFileCatalog
  metadata: {}


In [4]:
cat_keys = list(catalog._entries.keys())
catalog._entries

{'fgapo.carboscope.apo99_v2020': name: fgapo.carboscope.apo99_v2020
 container: xarray
 plugin: ['netcdf']
 driver: ['netcdf']
 description: APO fluxes from CarboScope inversion apo99_v2020
 direct_access: forbid
 user_parameters: []
 metadata: 
 args: 
   urlpath: /glade/work/mclong/sno-analysis/flux-products/fgapo_ocn.CarboScope.apo99_v2020.nc
   xarray_kwargs: 
     decode_times: False,
 'fgco2.MPI-SOM-FFN': name: fgco2.MPI-SOM-FFN
 container: xarray
 plugin: ['netcdf']
 driver: ['netcdf']
 description: An observation-based global monthly gridded sea surface pCO2 product from 1982 onward and its monthly climatology. Citation: Landschützer, P., Gruber, N., Bakker, D. C. E.: Decadal variations and trends of the global ocean carbon sink, Global Biogeochemical Cycles, 30, doi:10.1002/2015GB005359, 2016
 direct_access: forbid
 user_parameters: []
 metadata: 
 args: 
   urlpath: /glade/work/mclong/sno-analysis/flux-products/fgco2.MPI-SOM-FFN.v2018.monclim_2009-2018.nc
   xarray_kwargs: 
 

In [5]:
USER = os.environ['USER']
dirout = f'/glade/scratch/{USER}/sno-flux-products'
os.makedirs(dirout, exist_ok=True)

In [6]:
dso_grid = util.generate_latlon_grid(**config.config_dict["flux-dst-grid-kwargs"])[["area"]]
dso_grid

<xarray.Dataset>
Dimensions:  (lat: 180, lon: 360)
Coordinates:
  * lat      (lat) float64 -89.5 -88.5 -87.5 -86.5 -85.5 ... 86.5 87.5 88.5 89.5
  * lon      (lon) float64 -179.5 -178.5 -177.5 -176.5 ... 177.5 178.5 179.5
Data variables:
    area     (lat, lon) float64 1.079e+08 1.079e+08 ... 1.079e+08 1.079e+08

In [7]:
cluster, client = util.get_ClusterClient()
cluster.scale(12)

client

Connection method: Cluster object,Cluster type: dask_jobqueue.PBSCluster
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/mclong/calcs/proxy/8787/status,
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/mclong/calcs/proxy/8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.12.206.11:45502,Workers: 0
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/mclong/calcs/proxy/8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [ ]:
def get_varname_from_key(key, dsi):
    varname = key.split('.')[0]    
    if varname not in dsi.data_vars:
        varname = f'{varname}_ocn'
    assert varname in dsi.data_vars        
    return varname

In [8]:
year_range = 1999, 2018

time_units = 'days since 1990-01-01 00:00:00'
daily_time, daily_time_bnds = util.gen_daily_cftime_coord(year_range)        


dsets = {}
for key in cat_keys:
    dsi = catalog[key].to_dask()    
    varname = get_varname_from_key(key, dsi)
    is_climatology = 'climatology' in dsi.time.attrs
    
    if is_climatology:
        file_out = f'{dirout}/{key}.1x1.repeat_monclim.{year_range[0]}0101-{year_range[1]}1231.nc' 
    else:
        file_out = f'{dirout}/{key}.1x1.{year_range[0]}0101-{year_range[1]}1231.nc' 
        
    
    # if it's a monthly climatology, repeat it for the period of interest
    if is_climatology:
        data = np.concatenate([dsi[varname].data for i in range(year_range[0], year_range[1]+1)], axis=0)                
        monthly_time, monthly_time_bounds = util.gen_midmonth_cftime_coord(year_range)
        monthly_time_num = xr.DataArray(
            cftime.date2num(monthly_time, units=dsi.time.units),
            dims=('time'),
            attrs={'units': dsi.time.units, 'bounds': 'time_bnds'},
        )
        monthly_time_bounds_num = xr.DataArray(
            cftime.date2num(monthly_time_bounds, units=dsi.time.units),
            dims=('time', 'd2'),
        )        
        
        dims = dsi[varname].dims        
        dsi = dsi[[v for v in dsi.variables if 'time' not in dsi[v].dims]]
        
        dsi[varname] = xr.DataArray(data, dims=dims, 
                                    coords={'time': monthly_time_num},
                                    attrs=dsi[varname].attrs,
                                    name=varname,                                    
                                   )
        dsi['time_bnds'] = monthly_time_bounds_num
        dsi['time'] = monthly_time_num
        
    
    daily_time, daily_time_bnds = util.gen_daily_cftime_coord(year_range)        
    daily_time_num_data = cftime.date2num(daily_time, units=dsi.time.units)
        
    # interpolate
    drop_list = [v for v in dsi.data_vars if 'time' in dsi[v].dims and v != varname]
    dsi_daily = dsi.drop(drop_list).interp(time=daily_time_num_data)

    dsi_daily['time'] = daily_time
    dsi_daily.time.encoding['units'] = time_units    
    dsi_daily[daily_time.bounds] = daily_time_bnds    
    
    dsi_daily['time_components'] = util.gen_time_components_variable(daily_time)

    for v in ['area', 'lat', 'lon']:
        dsi_daily[v] = dso_grid[v]
    
    if dsi_daily[v].attrs['units'] == 'mol m-2 s-1':
        dsi_daily[v].attrs['units'] = 'mol/m^2/s'
        
    util.to_netcdf_clean(dsi_daily, file_out)
    dsets[key] = dsi_daily
    
dsi_daily

------------------------------
Writing /glade/scratch/mclong/sno-flux-products/fgapo.carboscope.apo99_v2020.1x1.19990101-20181231.nc
netcdf fgapo.carboscope.apo99_v2020.1x1.19990101-20181231 {
dimensions:
	lat = 180 ;
	lon = 360 ;
	time = 7305 ;
	d2 = 2 ;
	n_time_components = 6 ;
variables:
	double lat(lat) ;
		lat:units = "degrees_north" ;
		lat:long_name = "Latitude" ;
	double lon(lon) ;
		lon:units = "degrees_east" ;
		lon:long_name = "Longitude" ;
	float area(lat, lon) ;
		area:_FillValue = 9.96921e+36f ;
		area:units = "m^2" ;
		area:long_name = "area" ;
	float fgapo_ocn(time, lat, lon) ;
		fgapo_ocn:_FillValue = 9.96921e+36f ;
		fgapo_ocn:long_name = "Ocean-atmosphere apo flux" ;
		fgapo_ocn:units = "mol/m^2/s" ;
		fgapo_ocn:cell_methods = "lon: lat: sum" ;
	double time(time) ;
		time:bounds = "time_bnds" ;
		time:units = "days since 1999-01-01" ;
		time:calendar = "gregorian" ;
	double time_bnds(time, d2) ;
	float time_components(time, n_time_components) ;
		time_components:_Fil

<xarray.Dataset>
Dimensions:          (lat: 180, lon: 360, time: 7305, d2: 2, n_time_components: 6)
Coordinates:
  * lat              (lat) float64 -89.5 -88.5 -87.5 -86.5 ... 87.5 88.5 89.5
  * lon              (lon) float64 -179.5 -178.5 -177.5 ... 177.5 178.5 179.5
  * time             (time) object 1999-01-01 12:00:00 ... 2018-12-31 12:00:00
Dimensions without coordinates: d2, n_time_components
Data variables:
    area             (lat, lon) float64 1.079e+08 1.079e+08 ... 1.079e+08
    fgo2             (time, lat, lon) float64 nan nan nan nan ... nan nan nan
    time_bnds        (time, d2) object 1999-01-01 00:00:00 ... 2019-01-01 00:...
    time_components  (time, n_time_components) float64 1.999e+03 1.0 ... 0.0 0.0
Attributes:
    history:  created by https://github.com/matt-long/sno-analysis/tree/64621...

In [ ]:
dsi = dsets[cat_keys[0]]

region = xr.DataArray(['NET', 'SET'], dims=('region'), name='region')
masked_area = xr.concat([
    dsi.area.where((dsi.lat >= 20)),
    dsi.area.where((dsi.lat <= -20)),    
],
    dim=region,
)
masked_area.isel(region=0).plot()

In [ ]:
for key in cat_keys:
    print(key)
    
    dsi = dsets[key].sel(time='2009')
    v = get_varname_from_key(key, dsi)    
    assert dsi[v].units == 'mol/m^2/s'
    
    plt.figure()
    
    global_sum = ((dsi[v] * dsi.area).sum(['lat', 'lon'])).sum('time').values * 1e-12 * 365. * 86400.
    print(f'GLB {key}: {global_sum:04f} Tmol/yr')
    
    with xr.set_options(keep_attrs=True):
        dsi_region = (dsi * masked_area).sum(['lat', 'lon']) * 1e-12 * 365. * 86400.
        dsi_region[v].attrs['units'] = 'Tmol yr$^{-1}$'
    
    for region_name in dsi_region.region[i].values:   
        da = dsi_region[v].sel(region=region_name, drop=True)        
        regional_sum = da.sum('time').values
        
        da.plot(label=region_name)
        print(f'{region_name} {key}: {regional_sum} Tmol/yr')        
    plt.title(key)
    print()
    
axs[0].set_xticklabels([])
for ax in axs:
    ax.legend();
    

In [12]:
client.close()
cluster.close()